In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont, ImageFilter

In [3]:
IMG_PATH = 'Passport_Country.jpg'     # path ảnh
FONT_PATH = 'font/ocr-b-regular.ttf' # path font
TXT_PATH = 'Passport_Country_Gen.txt' # path file txt chứa data gen (mỗi dòng 88 ký tự)

# Folder để lưu ảnh với data gen
output_folder = 'Passport_Country_Gen'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Đọc txt
with open(TXT_PATH, 'r') as file:
    lines = file.readlines()

def process_image(line1, line2, image_rgb, mask, font, output_path):
    inpainted_image = cv2.inpaint(image_rgb, mask, 0, cv2.INPAINT_NS)
    cv2.imwrite(f'Passport_Country_background.jpg', cv2.cvtColor(inpainted_image, cv2.COLOR_RGB2BGR))
    inpainted_image_pil = Image.fromarray(inpainted_image)
    draw = ImageDraw.Draw(inpainted_image_pil)
    
    def draw_text_with_stretch(draw, text, position, font, stretch_factor, num_strokes):
        # Tạo ảnh tạm thời với nền trong suốt
        temp_image = Image.new('RGBA', (2530, 90), (0, 0, 0, 0))
        temp_draw = ImageDraw.Draw(temp_image)
        
        # Text lên hình ảnh tạm thời
        for dx in range(-num_strokes, num_strokes + 1):
            for dy in range(-num_strokes, num_strokes + 1):
                temp_draw.text((dx, dy), text, font=font, fill=(40, 40, 40, 255))
            
        # Dãn chiều ngang text 
        width, height = temp_image.size
        new_width = int(width * stretch_factor)
        stretched_image = temp_image.resize((new_width, height), Image.Resampling.BICUBIC)
        stretched_image = stretched_image.filter(ImageFilter.GaussianBlur(1)) # Thêm GausBlur làm mờ tí cho dúng thiệt V:

        # dãn xong sang RGB để paste vào ảnh chính
        stretched_image_rgb = stretched_image.convert("RGB")
        inpainted_image_pil.paste(stretched_image_rgb, position, stretched_image)
    
    # Vùng thứ nhất
    start_x1, start_y1 = 85, 1580  # Vùng thứ nhất
    draw_text_with_stretch(draw, line1, (start_x1, start_y1), font, stretch_factor=1.228, num_strokes=0)

    # Vùng thứ hai
    start_x2, start_y2 = 85, 1735  # Vùng thứ hai
    draw_text_with_stretch(draw, line2, (start_x2, start_y2), font, stretch_factor=1.228, num_strokes=0)

    inpainted_image_with_text = np.array(inpainted_image_pil)
    
    # Lưu ảnh
    cv2.imwrite(output_path, cv2.cvtColor(inpainted_image_with_text, cv2.COLOR_RGB2BGR))

In [4]:
# Đọc ảnh và tạo mask
image = cv2.imread(IMG_PATH)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

mask = np.zeros(image.shape[:2], np.uint8)
mask[1592:1660, 77:2690] = 255  # Vùng thứ nhất
mask[1750:1825, 77:2690] = 255  # Vùng thứ hai 

# Đọc font
font = ImageFont.truetype(FONT_PATH, 81)

In [5]:
# Xử lý file txt
for idx in range(0, len(lines), 2):
    line1 = lines[idx].strip()  # dòng đầu tiên
    line2 = lines[idx+1].strip() if (idx+1) < len(lines) else ""  # dòng thứ hai
    output_path = os.path.join(output_folder, f'Passport_Country_Gen_{idx//2 + 1}.jpg')
    process_image(line1, line2, image_rgb, mask, font, output_path)